In [3]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [4]:
val researchYear = "1998"

In [5]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [6]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [7]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [8]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                     578938|
+---------------------------+



In [9]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                     875773|
+---------------------------+



In [11]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(300,false)

+---------------------------------+------+
|mime                             |URLs  |
+---------------------------------+------+
|text/html                        |578938|
|image/gif                        |157703|
|image/jpeg                       |87418 |
|text/plain                       |23104 |
|text/pl                          |10065 |
|application/zip                  |4135  |
|im                               |2972  |
|unk                              |2003  |
|application/octet-stream         |1532  |
|application/x-gzip               |1294  |
|application/postscript           |1294  |
|application/pdf                  |969   |
|application/x-tar                |859   |
|audio/x-wav                      |592   |
|audio/x-pn-realaudio             |434   |
|application/x-dvi                |379   |
|application/msword               |366   |
|image/x-xbitmap                  |345   |
|audio/midi                       |341   |
|audio/mpeg3                      |219   |
|applicatio

In [12]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

102